## Initial Prompt : Design a Parking Lot System 

## Questions to be asked 
1. System Scope
    1. (Is this for a single Parking Lot or a generic system usable by multiple parking lots ?) Single Parking Lot : Singleton class, Multiple : Normal Class 
    2. Will this parking lot have multiple levels (floors) in it? Yes : Maintain a list in Parking lot called levels which has level objects in it 
2. Understand Classes and Enums
    1. Will there be multiple types of vehichles being parked here? (Enum Vehicle Type) and (Base Vehicle Abstract Class)
    2. Are we going to store the Receipts and reservation details ? (Receipt Class and Reservation Class)
    3. Are we going to have Parking Spots which are parkable by a specific vehicle type? (Yes, we are going to have)
3. Clarify the core features
    1. We are going to have check-in, checkout, real-time info and receipt generation as the features exposed to end user ?
    2. Are we going to calculate the fees based on the time spent? Is there a formula I need to use here?
    3. check-in : allocates parking spot to vehicle
    4. check-out : deallocates parking spot given to vehicle
    5. real-time info : displays vacant spots in the parking lot level wise
    6. receipt generation : Generates and stores receipt based on parking usage on check-out

## Basic LLD Design 
### Classes and Enums 
1. ParkingLot (Singleton)
   - Attributes: id, address, levels,reservationss
   - Method
       - : park_vehiclvehiclee() -reservation_id 
       - , unpark_vehicllevel_number, spot_numberi -> True/False
       - , get_availability
       - calc_price(start_time, end_time, vehicletype)()

2. Vehicle (Abstract)
   - Attributes: license_plate, vehicle_type
   - Concrete classes: Car, Motorcycle,
   - Methods : get_vehicle_type and get_license_plate Truck

3. VehicleType (Enum)
   - CAR, MOTORCYCLE, TRUCK

4. Level
   - Attributes: level_number, spots, available_(dict by v type)ehicle type)

       - park_vehiclevvehicle( -> spot_number, 
       - unpark_vehicle(spot_number) -> True/Falsev
       - get_availability() vailability()

5. Spot
   - Attributes: spot_npot_type, status, parked_vehic
       - e
   - Method -> True/Falses
       - parkailable()vehicle, -> spot_number 
       - unparkvehicle(), -> True/False remoReservationcle()

6. Receipt
   - Attributes: id, entry_time, exit_timlevel, e, vehicle, amount
       - init(),
       -  spot
   - Met,
       - print_receipt(),
       - get_parking_details()
hods: calculate_fee()

In [43]:
from datetime import datetime

In [44]:
from enum import Enum 

class VehicleType(Enum):
    MOTORCYCLE = 1
    CAR = 2
    TRUCK = 3


In [45]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    def __init__(self, license_plate :str, vehicle_type: 'VehicleType'):
        self.license_plate = license_plate
        self.vehicle_type = vehicle_type

    def get_license_plate(self):
        return self.license_plate

    def get_vehicle_type(self):
        return self.vehicle_type
    

In [46]:
class Car(Vehicle):
    def __init__(self, license_plate: str):
        super().__init__(license_plate, VehicleType.CAR)


In [47]:
class Motorcycle(Vehicle):
    def __init__(self, license_plate: str):
        super().__init__(license_plate, VehicleType.MOTORCYCLE)

In [48]:
class Truck(Vehicle):
    def __init__(self, license_plate: str):
        super().__init__(license_plate, VehicleType.TRUCK)

In [67]:
class Spot():
    def __init__(self, spot_number: int, vehicle_type : 'VehicleType'):
        self.spot_number = spot_number
        self.vehicle_type = vehicle_type
        self.parked_vehicle = None

    def is_available(self):
        return self.parked_vehicle is None

    def get_spot_type(self):
        return self.vehicle_type

    def park_vehicle(self, vehicle):

        self.parked_vehicle = vehicle 

        return self.spot_number

    def unpark_vehicle(self):

        print(self.parked_vehicle, self.spot_number)
        if self.is_available():
            print(self.parked_vehicle, self.spot_number)
            raise Exception('Spot is unreserved !')

        self.parked_vehicle  = None

        return True

In [78]:
from typing import Dict
class Level():
    def __init__(self, level_number : int, spot_mapping : Dict):

        self.level_number = level_number

        self.spots = []

        self.available_spot_counts = spot_mapping

        for spot_type, count in spot_mapping.items():
            for _ in range(count):
                spot = Spot(len(self.spots)+1, spot_type)
                self.spots.append(spot)

    def park_vehicle(self, vehicle):

        if self.available_spot_counts[vehicle.get_vehicle_type()] == 0:
            return None, None

        for spot in self.spots:
            if spot.get_spot_type() == vehicle.get_vehicle_type() and spot.is_available():
                spot_number = spot.park_vehicle(vehicle)
                self.available_spot_counts[spot.get_spot_type()]-=1
                return self.level_number, spot_number

        return None, None

    def unpark_vehicle(self, spot_number):

        for spot in self.spots:
            if spot.spot_number == spot_number:
                spot.unpark_vehicle()
                self.available_spot_counts[spot.get_spot_type()]+=1

        return True

    def get_available_spot_counts(self):
        for spot_type, value in self.available_spot_counts.items():
            print(f'{value} vacanies present in Level number {self.level_number} for spot type {spot_type}')
 
        

In [79]:
class ParkingLot():
    _instance = None

    def __init__(self):
        if ParkingLot._instance is not None:
            raise Exception('Only one instance is allowed in Singletone class. use get_instance to get access to the object')

        ParkingLot._instance = self
        self.levels = []
        self.reservation_id = 1
        self.reservations = {}

    @staticmethod
    def get_instance():
        if ParkingLot._instance is None:
            return ParkingLot()
        return ParkingLot._instance

    def add_level(self, level:Level):
        self.levels.append(level)

    def park_vehicle(self, vehicle: 'Vehicle'):

        for level in self.levels:
            level_number, spot_number = level.park_vehicle(vehicle)
            if level_number is not None:
                break

        if level_number is None:
            raise Exception('No empty slots in any level')

        reservation = Reservation(self.reservation_id, datetime.now(), level_number, spot_number)

        self.reservations[self.reservation_id]= reservation

        self.reservation_id+=1

        return reservation.reservation_id

    
    def unpark_vehicle(self, reservation_id: int):
        
        reservation = self.reservations[reservation_id]
        spot_number, level_number = reservation.spot_number, reservation.level_number
        reservation.end_time = datetime.now()
        reservation.print_receipt()

        for level in self.levels:
            if level_number == level.level_number:
                level.unpark_vehicle(spot_number)
                return True
        return False

    def display_availability(self):
        for level in self.levels:
            level.get_available_spot_counts()

        
        

In [80]:

class Reservation():

    def __init__(self, reservation_id : int, start_time : datetime, level_number : int, spot_number : int):
        self.reservation_id = reservation_id
        self.start_time = start_time
        self.level_number = level_number
        self.spot_number = spot_number
        self.end_time =  None
        self.fee = None

    def print_receipt(self):
        print(self.reservation_id, self.start_time, self.level_number, self.spot_number, self.end_time, self.fee)
    

In [81]:
def test_parking_lot_system():
    print("=== Testing Parking Lot System ===")
    
    # 1. Create parking lot instance
    print("\n1. Creating parking lot...")
    parking_lot = ParkingLot.get_instance()
    
    # 2. Add levels with spots
    print("\n2. Adding levels with spots...")
    # Level 1: 2 motorcycle spots, 3 car spots, 1 truck spot
    level1_spots = {
        VehicleType.MOTORCYCLE: 2,
        VehicleType.CAR: 3,
        VehicleType.TRUCK: 1
    }
    level1 = Level(1, level1_spots)
    parking_lot.add_level(level1)
    
    # Level 2: 1 motorcycle spot, 2 car spots, 0 truck spots
    level2_spots = {
        VehicleType.MOTORCYCLE: 1,
        VehicleType.CAR: 2,
        VehicleType.TRUCK: 0
    }
    level2 = Level(2, level2_spots)
    parking_lot.add_level(level2)
    
    # 3. Display initial availability
    print("\n3. Initial availability:")
    parking_lot.display_availability()
    
    # 4. Park vehicles
    print("\n4. Parking vehicles...")
    # Park 2 cars
    car1 = Car("CAR001")
    car2 = Car("CAR002")
    
    reservation1 = parking_lot.park_vehicle(car1)
    print(f"Parked car1 with reservation ID: {reservation1}")
    
    reservation2 = parking_lot.park_vehicle(car2)
    print(f"Parked car2 with reservation ID: {reservation2}")
    
    # Park 1 motorcycle
    motorcycle1 = Motorcycle("MOTO001")
    reservation3 = parking_lot.park_vehicle(motorcycle1)
    print(f"Parked motorcycle1 with reservation ID: {reservation3}")
    
    # Park 1 truck
    truck1 = Truck("TRUCK001")
    reservation4 = parking_lot.park_vehicle(truck1)
    print(f"Parked truck1 with reservation ID: {reservation4}")
    
    # 5. Display availability after parking
    print("\n5. Availability after parking:")
    parking_lot.display_availability()
    
    # 6. Unpark vehicles
    print("\n6. Unparking vehicles...")
    print("Unparking car1:")
    parking_lot.unpark_vehicle(reservation1)
    
    print("\nUnparking truck1:")
    parking_lot.unpark_vehicle(reservation4)
    
    # 7. Display final availability
    print("\n7. Final availability:")
    parking_lot.display_availability()
    
    # 8. Try to park when full
    print("\n8. Testing parking when full:")
    try:
        # Try to park more trucks than we have spots for
        truck2 = Truck("TRUCK002")
        truck3 = Truck("TRUCK003")
        
        reservation5 = parking_lot.park_vehicle(truck2)
        print(f"Parked truck2 with reservation ID: {reservation5}")
        
        # This should fail as we only had 1 truck spot and it's already taken
        reservation6 = parking_lot.park_vehicle(truck3)
        print(f"Parked truck3 with reservation ID: {reservation6}")
    except Exception as e:
        print(f"Exception as expected: {e}")
    
    print("\n=== Test completed ===")


In [82]:
test_parking_lot_system()

=== Testing Parking Lot System ===

1. Creating parking lot...

2. Adding levels with spots...

3. Initial availability:
2 vacanies present in Level number 1 for spot type VehicleType.MOTORCYCLE
3 vacanies present in Level number 1 for spot type VehicleType.CAR
1 vacanies present in Level number 1 for spot type VehicleType.TRUCK
1 vacanies present in Level number 2 for spot type VehicleType.MOTORCYCLE
2 vacanies present in Level number 2 for spot type VehicleType.CAR
0 vacanies present in Level number 2 for spot type VehicleType.TRUCK

4. Parking vehicles...
Parked car1 with reservation ID: 1
Parked car2 with reservation ID: 2
Parked motorcycle1 with reservation ID: 3
Parked truck1 with reservation ID: 4

5. Availability after parking:
1 vacanies present in Level number 1 for spot type VehicleType.MOTORCYCLE
1 vacanies present in Level number 1 for spot type VehicleType.CAR
0 vacanies present in Level number 1 for spot type VehicleType.TRUCK
1 vacanies present in Level number 2 for spo